In [ ]:
from azureml.core import Workspace,Datastore,Dataset

In [ ]:
ws = Workspace.from_config()
print(ws)

In [ ]:
# retrieve an existing datastore in the workspace by name
datastore  = ws.get_default_datastore()
#upload directory
dataset = Dataset.File.upload_directory(src_dir="data", 
                                        target=(datastore, "wq"))

In [ ]:
fd = Dataset.File.from_files(path=(datastore,'/'))
for fp in fd.to_path():
    print(fp)

In [ ]:
# create a TabularDataset from 1 file paths in datastore
#datastore_paths = [(datastore, "winequality")]
#create a tabular object from csv
custom_ds = Dataset.Tabular.from_delimited_files(path=(datastore,"/wq/winequality-red.csv"),separator=';', header=True,encoding='utf8')


In [ ]:
custom_ds.take(3).to_pandas_dataframe()

In [ ]:
#register dataset
custom_ds.register(workspace=ws,
                  name="winequality",
                  description="desc wq")

In [ ]:
ds = Dataset.get_by_name(ws,"winequality").to_pandas_dataframe()
ds.head(3)

In [ ]:
#create train/test sets

from sklearn.model_selection import train_test_split
y = ds['quality']
X = ds.drop('quality',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.columns

In [ ]:
train_dt = X_train
train_dt['quality'] = y_train
train_dt.columns


In [ ]:
test_dt = X_test
test_dt['quality'] = y_test
test_dt.columns

In [ ]:
train_dt.to_csv("data/wine-quality-train.csv")
test_dt.to_csv("data/wine-quality-test.csv")

In [ ]:
#datastore.upload_files(files=['data/wine-quality-train.csv','data/wine-quality-test.csv'])
#upload train & test sets to datastore
dataset2 = Dataset.File.upload_directory(src_dir="data", 
                                        target=(datastore, "wq"))

In [ ]:
trcsvpath = [(datastore,'/wq/wine-quality-train.csv')]
tstcsvpath = [(datastore,'/wq/wine-quality-test.csv')]

trds = Dataset.Tabular.from_delimited_files(path=trcsvpath)
trds = trds.register(ws,'wine-quality-train')
tstds = Dataset.Tabular.from_delimited_files(path=tstcsvpath)
tstds = trds.register(ws,'wine-quality-test')

In [ ]:
Dataset.get_by_name(ws,'wine-quality-train').to_pandas_dataframe().head(3)